In [ ]:
'''
Section 1 Project (s1p)
▫︎ overview : 게임회사의 데이터 팀 합류, 다음 분기에 어떤 게임을 설계해야 할까?, 기존 데이터에 기반해 인사이트 도출 및 의사결정 진행
▫︎ data : https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv
  - description
    • 'Name' : 게임명
    • 'Platform' : 게임이 지원되는 플랫폼명
    • 'Year' : 게임 출시 연도
    • 'Genre' : 게임 장르
    • 'Publisher' : 게임 제작 회사명
    • 'NA_Sales' : 북미지역 출고량
    • 'EU_Sales' : 유럽지역 출고량
    • 'JP_Sales' : 일본지역 출고량
    • 'Other_Sales' : 기타지역 출고량
▫︎ Goal of Project : 다음 분기에 어떤 게임을 설계해야 할까?
▫︎ Reference Keywords
  - Sprint 1 : EDA, Feature Engineering, Data Manipulation, Data Visualization
  - Sprint 2 : Hypothesis Test, CI, Bayesian, AB Test
  - Sprint 3 : DR, PCA, Clustering, GD
▫︎ Mission
  1. 지역에 따라서 선호하는 게임 장르가 다를까?
  2. 연도별 게임의 트렌드가 있을까?
  3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스
▫︎ Others
  - data를 통해 도출될 수 있는 다른 분석목표를 수립하고 진행해 본다.
  - 추가 질문에 대한 답을 찾아본다.
▫︎ Output
  - presentation video (8 min)
  - slide
  - script
''' 

In [ ]:
'''
s1p Design
▫︎ term definition
  - "출고량 = 매출량" 으로 본다. 용어가 다르나, 여기서는 sales(매출량)을 출고량으로 표현한 것 같다.
  - 'Platform' = "device(기기)" 로 본다. 여기서는 PC, 각 콘솔명, 애뮬레이터 명을 독립적으로 표기했고 이를 플랫폼으로 표현한 것 같다.
  - 'Publisher' = "productor(제작사)" 로 본다. 엄밀히 제작사와 배급사는 같지 않을 수 있지만, 여기서는 모두 퍼블리셔로 표현한 것 같다.
▫︎ EDA direction
  - dataframe(df)의 인덱스를 정해야 한다. 이는 분석하고자 하는 방향에 따라 가변적일 수 있다.
  - df를 가공시 원본에서 copy본을 생성해야 한다.
  - 각 column의 결측치와 이상치를 확인해야 한다. 그리고 제거해도 되는지 판단해야 한다.
  - 각 column의 data 특성을 파악해야 한다. 그리고 dtype을 확인해야 한다.
    • 'Name' : 중복값 확인 필요, 중복이 있다면 왜 존재하는지 파악 ('Platform', 'Year', 'Genre', 'Publisher'를 비교) / dtype 'str'
    • 'Platform' : categorizing 필요 ('Platform' 별로 df를 따로 저장) / dtype 'str'
    • 'Year' : 최소값과 최대값 확인 필요, 최소값이 언제인지(이상치 여부 파악), 최대값은 언제인지(현재 시점 2022 초과면 이상치로 판단) / dtype 'integer'
    • 'Genre' : categorizing 필요 ('Genre' 별로 df를 따로 저장) / dtype 'str'
    • 'Publisher' : categorizing 필요 ('Publisher' 별로 df를 따로 저장) / dtype 'str'
    • 'Sales' : 총 4개 column의 값들에 대한 단위 기준을 마련해야 함 / dtype 'float'
       ✓ 여기서는 floating values 들을 편의상 모두 국제 기준 통화(USD)를 의미하는 것으로 간주한다.
       ✓ 표기된 단위는 편의상 Million으로 간주한다. (예, 0.04이면 $0.04M = $40,000)
       ✓ null값이 아닌 '0'인 값들은 매출이 없음을 의미하는데, 여기서는 편의상 해당 지역에서는 출시되지 않은 것으로 간주한다.
  - 그 외 EDA에 대한 추가적인 사항이 나오면 기록하고 실행한다.
▫︎ Mission solving
  1. 지역에 따라서 선호하는 게임 장르가 다를까?
    • 'Genre' 별로 4개 지역의 출고량 합을 각각 구한다. (어느 지역에서 이 장르의 출고량이 높다/낮다를 확인)
    • 'Publisher'와 'Genre'의 상관관계를 파악하면 4개 지역별로 출고가 높은 게임 제작사도 볼 수 있을 것이다. (게임 제작사가 로컬 편향인지 글로벌 기업인지 확인)
  2. 연도별 게임의 트렌드가 있을까?
    • 'Year'와 'Genre'의 상관관계를 분석한다. (연도별로 많이/적게 출시된 게임장르가 있는지 확인)
    • 'Year'와 'Platform'의 상관관계를 분석한다. (연도별로 플랫폼이 어떻게 변화하는지 확인)
    • 'Year', 'Genre', 'Platform'과 각 지역 출고량의 상관관계를 분석한다. (어느 지역에서 언제 어떤 게임이 많이 출고되었는지 확인)
  3. 출고량이 높은 게임에 대한 분석 및 시각화 프로세스
    • 16,000개가 넘는 'Name'들을 하나씩 보여주는 것은 비효율적이다.
    • 따라서 이 부분은 'Genre' 별로 각 지역별 출고량의 합을 통해서 1차적인 분석을 제시한다.
    • 'Genre' 별로 가장 높은 출고량을 기록한 'Name'을 확인한다.
    • 이를 시각화하기 위한 어떤 챠트가 효과적인지 대해서 고민해야 한다.
▫︎ Conclusion
  - Mission solving을 통한 인사이트와 결론을 도출한다.
  - 더 분석하면 좋을 내용이 있는지 확인한다.
'''

In [ ]:
# EDA

In [2]:
# pd.read_csv()

import pandas as pd
df = pd.read_csv("https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/datasets/vgames2.csv")

In [3]:
# df 살펴보기

df.head()

,Unnamed: 0,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,1,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,2,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,4,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04


In [4]:
# 0번째 컬럼인 'Unnamed: 0'을 iloc 함수를 이용해 삭제
# df_basic으로 저장
df_basic = df.iloc[:,1:]
df_basic.tail()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03
16597,The King of Fighters '95,PS,1996.0,Fighting,Sony Computer Entertainment,0,0,0.16,0.01


In [5]:
df_basic.shape

(16598, 9)

In [6]:
df_basic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16598 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16598 non-null  object 
 1   Platform     16598 non-null  object 
 2   Year         16327 non-null  float64
 3   Genre        16548 non-null  object 
 4   Publisher    16540 non-null  object 
 5   NA_Sales     16598 non-null  object 
 6   EU_Sales     16598 non-null  object 
 7   JP_Sales     16598 non-null  object 
 8   Other_Sales  16598 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.1+ MB


In [7]:
# df_basic에서 'Year', 'Genre', 'Publisher'에 결측치가 있는 것을 확인
# dtype 변경이 필요한 컬럼 확인 ('Year': int, 'NA_Sales': float, 'EU_Sales': float, 'JP_Sales': float, 'Other_Sales': float)

In [8]:
# 결측치 위치 확인 : df[df['A'].isnull()]
# 'Year'의 결측치 위치 확인 (총 271행)
df_basic[df_basic['Year'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
273,Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
358,Majesty 2: The Fantasy Kingdom Sim,X360,NaN,Simulation,Unknown,0.03,0,0,0
430,Famista 64,N64,NaN,Sports,Namco Bandai Games,0,0,0.17,0.03
...,...,...,...,...,...,...,...,...,...
16240,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0,0,0
16275,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0M,0.03
16278,Madden NFL 11,Wii,NaN,Sports,Unknown,0.7,0,0,50K
16425,Yoostar on MTV,X360,NaN,Misc,Unknown,0.06,0,0,0


In [9]:
# 'Genre'의 결측치 위치 확인 (총 50행)
df_basic[df_basic['Genre'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
50,Ape Escape: On the Loose,PSP,2005.0,NaN,Sony Computer Entertainment,0.52,0.01,0.12,0.05
408,NBA Ballers,XB,2004.0,NaN,Midway Games,0.37,0.11,0,0.02
1328,NFL Street,XB,2004.0,NaN,Electronic Arts,0.49,0.15,0,0.02
1582,SingStar Singalong With Disney,PS2,2008.0,NaN,Sony Computer Entertainment,0,0.04,0,0.57
1971,Pokemon X/Pokemon Y,3DS,2013.0,NaN,Nintendo,5170K,4.05,4.34,0.79
2043,NASCAR Racing,PS,1996.0,NaN,Pioneer LDC,0.23,0.16,0,0.03
2497,Jurassic Park III: Island Attack,GBA,2001.0,NaN,Konami Digital Entertainment,0.13,0.05,0,0
3353,The Godfather: Dons Edition,PS3,2007.0,NaN,Electronic Arts,0.25,0.02,0,0.03
3363,Ski-Doo Snowmobile Challenge,X360,2009.0,NaN,Valcon Games,0.03,0,0,0
3470,Batman Beyond: Return of the Joker,N64,2000.0,NaN,Ubisoft,0.07,0.02,0,0


In [10]:
# 'Publisher'의 결측치 위치 확인 (총 58행)
df_basic[df_basic['Publisher'].isnull()]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
272,UK Truck Simulator,PC,2010.0,Simulation,NaN,0,0.03,0,0
1313,All Grown Up!: Game Boy Advance Video Volume 1,GBA,2004.0,Misc,NaN,0.17,0.06,0,0
1560,Teenage Mutant Ninja Turtles: Game Boy Advance...,GBA,2004.0,Misc,NaN,0.12,0.04,0,0
1789,Nicktoons Collection: Game Boy Advance Video V...,GBA,2004.0,Misc,NaN,0.16,0.06,0,0
1879,Monster Hunter Frontier Online,PS3,NaN,Role-Playing,NaN,0,0,0.05,0
1961,Nicktoons Collection: Game Boy Advance Video V...,GBA,2004.0,Misc,NaN,0.46,0.17,0,0.01
2070,World of Tanks,X360,NaN,Shooter,NaN,0,0.03,0,0
2970,Chou Soujuu Mecha MG,DS,NaN,Simulation,NaN,0,0,0.06,0
3184,Action Man-Operation Extreme,PS,NaN,Action,NaN,0.05,0.03,0,0.01
3412,Dance with Devils,PSV,2016.0,Action,NaN,0,0,0.01,0


In [11]:
# 'Year' 또는 'Genre' 또는 'Publisher'의 결측치 위치 확인 (총 357행)
df_basic[(df_basic['Year'].isnull()) | (df_basic['Genre'].isnull()) | (df_basic['Publisher'].isnull())]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
31,Rock Band,X360,NaN,Misc,Electronic Arts,1.93,0.34,0,0.21
50,Ape Escape: On the Loose,PSP,2005.0,NaN,Sony Computer Entertainment,0.52,0.01,0.12,0.05
109,Yu-Gi-Oh! 5D's Wheelie Breakers (JP sales),Wii,NaN,Racing,Unknown,0,0,0.02,0
272,UK Truck Simulator,PC,2010.0,Simulation,NaN,0,0.03,0,0
273,Jewel Link Chronicles: Mountains of Madness,DS,NaN,Puzzle,Avanquest,0,0.06,0,0.01
...,...,...,...,...,...,...,...,...,...
16425,Yoostar on MTV,X360,NaN,Misc,Unknown,0.06,0,0,0
16431,Cars: Mater-National Championship,X360,2007.0,NaN,THQ,0.13,0,0,0.01
16432,Bejeweled 3,X360,NaN,Puzzle,Unknown,0.13,0,0,0.01
16555,Apache: Air Assault,PC,2010.0,NaN,Just Flight,0,0.03,0,0.01


In [12]:
# 'Year' 그리고 'Genre' 그리고 'Publisher'의 결측치 위치 확인 (없음)
df_basic[(df_basic['Year'].isnull()) & (df_basic['Genre'].isnull()) & (df_basic['Publisher'].isnull())]

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales


In [ ]:
# 16,587행의 데이터 중 357행의 결측치를 확인했다.
# 결측치의 비중은 전체의 약 2.15%에 해당한다. (아직 결측치를 모두 제거할지, 아니면 대체값을 찾을지 판단 보류)

In [13]:
# 'Name'의 중복 데이터 존재 여부 확인
df_basic_dup = df_basic.duplicated(['Name'])
df_basic_dup

0        False
1        False
2        False
3        False
4        False
         ...  
16593     True
16594    False
16595     True
16596     True
16597     True
Length: 16598, dtype: bool

In [14]:
# 'Name'에 중복 데이터가 존재한다.
# value_counts() 라는 숫자를 세주는 함수로 세부 내용을 확인해 본다.
print(df_basic['Name'].value_counts())

Need for Speed: Most Wanted            12
FIFA 14                                 9
Ratatouille                             9
Madden NFL 07                           9
LEGO Marvel Super Heroes                9
                                       ..
Saki: Achiga-Hen Portable               1
Shining Force Feather                   1
Model * Oshare Audition: Dream Girl     1
Twisted Metal 2                         1
Rainbow Islands: Revolution             1
Name: Name, Length: 11493, dtype: int64


In [15]:
df_basic.Name.value_counts()

Need for Speed: Most Wanted            12
FIFA 14                                 9
Ratatouille                             9
Madden NFL 07                           9
LEGO Marvel Super Heroes                9
                                       ..
Saki: Achiga-Hen Portable               1
Shining Force Feather                   1
Model * Oshare Audition: Dream Girl     1
Twisted Metal 2                         1
Rainbow Islands: Revolution             1
Name: Name, Length: 11493, dtype: int64

In [16]:
df_basic['Name'].nunique(dropna=False)

11493

In [17]:
# 'Year', 'Genre', 'Publisher' 중 결측치가 하나라도 있는 행은 모두 357행 이다.
# 'Name' 에 중복값이 11,493개 존재한다. 그러나 'Name' 과 연계되어 있는 'Platform', 'Year'를 확인해 보면 다른 값들이 들어있음을 알수 있어 'Name'의 중복값은 제거하지 않기로 한다.
# 따라서, 원활한 데이터분석을 위해 결측치가 들어있는 357행의 데이터만 삭제하기로 한다.

In [19]:
df_basic_drop = df_basic.dropna()
df_basic_drop

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
0,Candace Kane's Candy Factory,DS,2008.0,Action,Destineer,0.04,0,0,0
1,The Munchables,Wii,2009.0,Action,Namco Bandai Games,0.17,0,0,0.01
2,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010.0,Adventure,Alchemist,0,0,0.02,0
3,Deal or No Deal: Special Edition,DS,2010.0,Misc,Zoo Games,0.04,0,0,0
4,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010.0,Platform,D3Publisher,0.12,0.09,0,0.04
...,...,...,...,...,...,...,...,...,...
16593,Ice Age 2: The Meltdown,GC,2006.0,Platform,Vivendi Games,0.15,0.04,0,0.01
16594,Rainbow Islands: Revolution,PSP,2005.0,Action,Rising Star Games,0.01,0,0,0
16595,NBA 2K16,PS3,2015.0,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13
16596,Toukiden: The Age of Demons,PSV,2013.0,Action,Tecmo Koei,0.05,0.05,0.25,0.03


In [ ]:
# df_basic_drop 변수에 357행의 결측치 데이터를 제거한 값을 넣었다.
# 이제 각 컬럼별로 dtype을 맞춰준다.

In [22]:
df_basic_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB


In [24]:
# dtype 변경이 필요한 컬럼 확인 ('Year': int, 'NA_Sales': float, 'EU_Sales': float, 'JP_Sales': float, 'Other_Sales': float)
df_basic_drop = df_basic_drop.astype({'Year':'int'})
df_basic_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 0 to 16597
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Name         16241 non-null  object
 1   Platform     16241 non-null  object
 2   Year         16241 non-null  int64 
 3   Genre        16241 non-null  object
 4   Publisher    16241 non-null  object
 5   NA_Sales     16241 non-null  object
 6   EU_Sales     16241 non-null  object
 7   JP_Sales     16241 non-null  object
 8   Other_Sales  16241 non-null  object
dtypes: int64(1), object(8)
memory usage: 1.2+ MB


In [27]:
df_basic_drop = df_basic_drop.astype({'NA_Sales':'float'})
df_basic_drop.info()

ValueError: ignored

In [ ]:
# 출고량 데이터에 K, M등의 문자열이 포함되어 있는 데이터들이 있어 astype()함수로 한번에 데이터 타입이 변경되지 않는다.
# 문자열이 포함되어 있는 데이터들을 먼저 처리해야 한다.
# 표기된 단위는 편의상 Million으로 간주한다. (예, 0.04이면 $0.04M = $40,000)

In [71]:
df_basic_drop_k = df_basic_drop[(df_basic_drop['NA_Sales'].str.contains('K')) | (df_basic_drop['EU_Sales'].str.contains('K')) | (df_basic_drop['JP_Sales'].str.contains('K')) | (df_basic_drop['Other_Sales'].str.contains('K'))]
df_basic_drop_k

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
10,Ford Racing,PS,2001,Racing,Empire Interactive,480K,0.33M,0K,0.06
45,Resident Evil: Revelations 2,PS3,2015,Action,Capcom,60K,0.08,0.16,0.03
89,Bugs Bunny: Lost in Time,PS,1999,Platform,Infogrames,0.22,0.15,0K,0.03
90,Tom Clancy's Ghost Recon: Predator,PSP,2010,Shooter,Ubisoft,0.07,0.02,0K,0.02
128,Rozen Maiden: Gebetgarten,PS2,2007,Fighting,Taito,0,0K,0.01,0
...,...,...,...,...,...,...,...,...,...
16476,NBA Ballers: Chosen One,PS3,2008,Sports,Midway Games,0.12,0,0K,0.01
16492,Mortal Kombat,PSV,2012,Fighting,Warner Bros. Interactive Entertainment,470K,0.18,0,0.12
16525,Planet Monsters,GBA,2001,Action,Titus,0.01,0K,0,0
16578,Gex: Enter the Gecko,PS,1998,Platform,BMG Interactive Entertainment,320K,0.22,0,0.04


In [73]:
# K 문자열부터 제거
df_basic_drop_k['NA_Sales'] = df_basic_drop_k['NA_Sales'].str.strip('K')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [74]:
df_basic_drop_k['EU_Sales'] = df_basic_drop_k['EU_Sales'].str.strip('K')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [75]:
df_basic_drop_k['JP_Sales'] = df_basic_drop_k['JP_Sales'].str.strip('K')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
df_basic_drop_k['Other_Sales'] = df_basic_drop_k['Other_Sales'].str.strip('K')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
df_basic_drop_k

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
10,Ford Racing,PS,2001,Racing,Empire Interactive,480,0.33M,0,0.06
45,Resident Evil: Revelations 2,PS3,2015,Action,Capcom,60,0.08,0.16,0.03
89,Bugs Bunny: Lost in Time,PS,1999,Platform,Infogrames,0.22,0.15,0,0.03
90,Tom Clancy's Ghost Recon: Predator,PSP,2010,Shooter,Ubisoft,0.07,0.02,0,0.02
128,Rozen Maiden: Gebetgarten,PS2,2007,Fighting,Taito,0,0,0.01,0
...,...,...,...,...,...,...,...,...,...
16476,NBA Ballers: Chosen One,PS3,2008,Sports,Midway Games,0.12,0,0,0.01
16492,Mortal Kombat,PSV,2012,Fighting,Warner Bros. Interactive Entertainment,470,0.18,0,0.12
16525,Planet Monsters,GBA,2001,Action,Titus,0.01,0,0,0
16578,Gex: Enter the Gecko,PS,1998,Platform,BMG Interactive Entertainment,320,0.22,0,0.04
